In [1]:
import numpy as np
import pandas as pd

from fbprophet import Prophet

from keras.layers import Dense, Activation, GRU, Dropout
from keras.models import Sequential

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/env_zipline/lib/python3.5/site-packages/tensorflow/python/framework/ops.py:923: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/home/ubuntu/anaconda3/envs/env_zipline/lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  return _inspect.getargspec(target)


In [2]:
etf_frame = pd.read_csv('/home/ubuntu/csvs/etfs_pivot.csv')
etfs = etf_frame.columns.tolist()

In [3]:
etfs.pop(0)

'date'

In [4]:
etf_frame

,date,AAXJ,ACIM,ACWI,ACWV,ACWX,ADRD,ADRE,AFK,AGG,...,XTN,YANG,YCS,YINN,YMLI,YMLP,YYY,ZMLP,ZROZ,ZSL
0,2014-03-17,51.8998,55.3618,51.1969,56.4986,39.9088,20.5917,31.1902,27.6914,95.1937,...,41.4988,741.9276,65.7700,18.0676,14.6311,53.9279,15.0075,25.0558,79.4274,37.1900
1,2014-03-18,52.2619,55.6217,51.5374,56.8371,40.1901,20.7505,31.5912,27.7092,95.3084,...,41.7226,704.7079,65.3100,18.8990,14.7005,53.8652,15.0523,25.2310,79.7748,38.2300
2,2014-03-19,51.3565,55.6217,51.0536,56.3470,39.6539,20.5163,31.3451,27.2374,94.8848,...,41.4290,737.4106,66.7300,18.0382,14.5478,53.5998,14.9886,25.0308,79.1755,39.2064
3,2014-03-20,51.1120,55.6217,51.1700,56.2667,39.6012,20.4829,31.4864,27.5400,94.8142,...,41.3328,745.2790,66.5800,17.9404,14.5339,53.6893,14.9440,25.1371,79.4534,40.1050
4,2014-03-21,51.4742,55.2919,51.0536,56.1151,39.5661,20.4949,31.5548,27.9050,94.9466,...,41.3183,682.5782,66.2200,19.3881,14.5061,53.6893,14.9695,25.1034,80.3221,40.1450
5,2014-03-22,51.4742,55.2919,51.0536,56.1151,39.5661,20.4949,31.5548,27.9050,94.9466,...,41.3183,682.5782,66.2200,19.3881,14.5061,53.6893,14.9695,25.1034,80.3221,40.1450
6,2014-03-23,51.4742,55.2919,51.0536,56.1151,39.5661,20.4949,31.5548,27.9050,94.9466,...,41.3183,682.5782,66.2200,19.3881,14.5061,53.6893,14.9695,25.1034,80.3221,40.1450
7,2014-03-24,51.9269,55.1304,50.9908,56.2756,39.6715,20.4641,31.8920,27.7715,95.0878,...,41.1997,647.6624,66.2500,20.4153,14.4367,53.2418,14.9122,25.0057,81.4687,41.4350
8,2014-03-25,52.3344,55.3084,51.3851,56.5789,40.1373,20.6669,32.1837,27.7804,94.9907,...,41.3358,633.6470,66.3700,20.8065,14.4235,53.2717,14.9635,25.0121,81.0343,41.3350
9,2014-03-26,52.3615,55.3084,51.2148,56.4630,40.1197,20.6908,32.1472,27.9584,95.2907,...,40.6702,627.7457,65.9900,21.0315,14.4950,53.1524,14.9507,25.0438,81.7132,42.2800


In [5]:
train_date = pd.date_range(start='2014-03-17', end='2017-03-15')
macro_date = pd.date_range(start='2016-03-17', end='2017-03-15')
val_date = pd.date_range(start='2017-03-17', end='2018-03-15')
test_date = pd.date_range(start='2018-03-17', end='2019-03-15')

In [6]:
macro = pd.read_csv('/home/ubuntu/csvs/macro_pivot.csv')

In [7]:
etf_frame = etf_frame.set_index('date')
macro = macro.set_index('date')

etf_frame.index = pd.to_datetime(etf_frame.index)
macro.index = pd.to_datetime(macro.index)

In [8]:
predictions = pd.DataFrame(index=test_date)

In [9]:
X_frame = macro[macro.index.isin(macro_date)]
X_array = X_frame.values
X = np.reshape(X_array, (X_array.shape[0], X_array.shape[1], 1))

In [10]:
X.shape

(364, 28779, 1)

In [11]:
def make_prediction(etf):
    single_etf = etf_frame[etf]
    single_etf = single_etf.reset_index()
    single_etf.columns = ['ds','y']
    
    arima = Prophet()
    arima.fit(single_etf[single_etf['ds'].isin(train_date)])
    future = arima.make_future_dataframe(periods=len(val_date))
    forecast = arima.predict(future)
    
    residuals = single_etf[single_etf['ds'].isin(val_date)]['y'] - forecast[forecast['ds'].isin(val_date)]['yhat']
    y = residuals
    y = np.reshape(y, (y.shape[0], 1))
    gru_model = Sequential()
    
    gru_model.add(GRU(units=100, input_shape=(28779,1), return_sequences=True))
    gru_model.add(Dropout(0.2))
    gru_model.add(GRU(units=50, return_sequences=False))
    gru_model.add(Dropout(0.2))
    gru_model.add(Dense(1))
    gru_model.add(Activation('linear'))
    
    gru_model.compile(loss='mse', optimizer='rmsprop')
    gru_model.fit(x=X,y=y, batch_size=273, epochs = 1, validation_split=0.25)
    
    new_future = arima.make_future_dataframe(periods=1000)
    new_forecast = arima.predict(new_future)
    
    new_X_array = macro[macro.index.isin(val_date)].values
    new_X = np.reshape(X_array, (X_array.shape[0], X_array.shape[1], 1))
    
    new_residuals = gru_model.predict(new_X)
    arima_prediction = new_forecast[new_forecast['ds'].isin(test_date)]['yhat']
    
    new_residuals = new_residuals.reshape(new_residuals.shape[1], new_residuals.shape[0])[0]
    
    prediction = np.array(arima_prediction) + np.array(new_residuals)
    
    return prediction

In [12]:
etf_risky = ['BIB', 'BRZU', 'CURE', 'DRN', 'EDC', 'FAS', 'FINU', 'GOEX', 'INDL','JNUG', 'JPNL', 'KOLD', 'LBJ',
             'MIDU', 'NUGT', 'QLD', 'RETL', 'ROM','SCO', 'SOXL', 'SPXL', 'SVXY', 'TECL', 'TMF', 'TNA', 'TQQQ',
             'UBR','UDOW', 'UMDD', 'UPRO', 'URTY', 'USD', 'UWM', 'XPP', 'YINN']

for etf in etf_risky:    
    prediction = make_prediction(etf)
    
    predictions[etf] = prediction
    predictions.to_csv('arima_gru_predictions.csv')

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ubuntu/anaconda3/envs/env_zipline/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/ubuntu/anaconda3/envs/env_zipline/lib/python3.5/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


Train on 273 samples, validate on 91 samples
Epoch 1/1
273/273 [==============================] - 342s 1s/step - loss: 228.8383 - val_loss: nan


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Train on 273 samples, validate on 91 samples
Epoch 1/1
273/273 [==============================] - 294s 1s/step - loss: 5.6824 - val_loss: nan


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Train on 273 samples, validate on 91 samples
Epoch 1/1
273/273 [==============================] - 270s 989ms/step - loss: 73.2231 - val_loss: nan


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Train on 273 samples, validate on 91 samples
Epoch 1/1
273/273 [==============================] - 261s 955ms/step - loss: 123.5859 - val_loss: nan


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Train on 273 samples, validate on 91 samples
Epoch 1/1
273/273 [==============================] - 255s 936ms/step - loss: 57.3454 - val_loss: nan


In [16]:
len(etf_risky)

35

In [14]:
new_residuals.shape

NameError: name 'new_residuals' is not defined